#### AUC and confusion matrix

The math: straightforward and ignored

In [65]:
import numpy as np 

def conf_matrix(y_pred, y_true, granularity = 0.01):  #shape(n, 1)
    y_pred = y_pred.reshape(-1, 1)
    y_true = y_true.reshape(-1, 1).astype(bool)
    threshold = np.arange(0, 1.001, step=granularity).reshape(1, -1) 
    picked = y_pred <= threshold
    tp = np.sum(picked & y_true, axis=0)
    fp = np.sum(picked, axis=0) - tp
    tn = np.sum(~picked & ~y_true, axis=0)
    fn = np.sum(~picked & y_true, axis=0)
    return threshold, tp, fp, tn, fn, picked

t, tp, fp, tn, fn, _ = conf_matrix(np.array([0.3, 0.2, 0.8]), np.array([1,0,1]))

precision = tp / (tp + fp)
recall = tp / (tp + fn)
fpr = fp / (fp + tn)
f1 = 2* precision * recall / (recall + precision)

def get_auc(tpr, fpr):
    prev_fpr = 0
    prev_tpr = 0
    ret = 0
    for fpr_p, tpr_p in zip(fpr, tpr):
        d = fpr_p - prev_fpr
        h = (tpr_p + prev_tpr) / 2
        ret += d*h
        prev_tpr, prev_fpr = tpr_p, fpr_p

    return ret

C:\Users\beati\AppData\Local\Temp\ipykernel_47216\1088467669.py:16: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
C:\Users\beati\AppData\Local\Temp\ipykernel_47216\1088467669.py:19: RuntimeWarning: invalid value encountered in divide
  f1 = 2* precision * recall / (recall + precision)


In [76]:
def my_auc(y_pred, y_true, epsilon = 0.001):
    t, tp, fp, tn, fn, picked = conf_matrix(y_pred, y_true)
    recall = tp / (tp + fn + epsilon)
    fpr = fp / (fp + tn + epsilon)
    return get_auc(recall, fpr)

y_p = np.random.rand(500)
y = np.random.randint(low=0, high=2, size=500)

print(f'my auc: {my_auc(y_p, y)}')

from sklearn import metrics
def sklearn_auc(y_p, y):
    fpr, tpr, _ = metrics.roc_curve(y, y_p)
    return metrics.auc(fpr, tpr)
print(f'sklearn auc: {sklearn_auc(y_p, y)}')

my auc: 0.504349781900339
sklearn auc: 0.4961924040871409
